In [1]:
import pandas as pd
import math

In [2]:
df_supermarket_products = pd.read_csv('../../data/aryan_pandas/supermarket_products.csv',encoding='cp1252')
df_supermarket_location = pd.read_csv('../../data/aryan_pandas/establishment_catalonia.csv')
df_customer_info = pd.read_csv('../../data/aryan_pandas/customers.csv')
df_cust_location = pd.read_csv('../../data/aryan_pandas/location.csv')
df_cust_loc_mapping = pd.read_csv('../../data/aryan_pandas/customer_location.csv')

In [3]:
# Products available in different stores

df_supermarket_products.head()

,store_id,store_name,product_id,product_name,product_price,manufacture_date,expiry_date,quantity
0,cens-0820210007-3653,ESCLAT-BON PREU,105.1.1,Amul Kool (Kesar),23,01 Mar 2024,28 Aug 2024,21
1,cens-0820210007-3653,ESCLAT-BON PREU,46.1.9,PARLE Krack Jack Crackers Sweet & Salty (Mitha...,4,01 Feb 2024,27 Sep 2024,54
2,cens-0820210007-3653,ESCLAT-BON PREU,33.1.10,RUNutz Almonds (Badam) (1 kg),"1,025",01 Feb 2024,29 Jul 2024,7
3,cens-0820210007-3653,ESCLAT-BON PREU,28.1.17,"Utkal's Kasper HMT Rice (Full Grain, Steam) (...",826,01 Feb 2024,27 Oct 2024,68
4,cens-0820210007-3653,ESCLAT-BON PREU,58.1.15,"Nescafe Classic Coffee Powder, 100% Pure Insta...",17,01 Jul 2023,29 Jan 2025,15


In [4]:
df_customer = df_customer_info.merge(df_cust_loc_mapping, on='customer_id')
df_customer = df_customer.merge(df_cust_location, on='location_id')
df_customer.head()

,customer_id,customer_name,email_id,location_id,postal_code,latitude,longitude
0,1,Debra Kelly,sony.sth8@gmail.com,21788,8940,41.3500,2.0833
1,326,Dorothy Ortega,sony.shrestha@estudiantat.upc.edu,21788,8940,41.3500,2.0833
2,450,Yvonne Crawford,sony.shrestha@estudiantat.upc.edu,21788,8940,41.3500,2.0833
3,2,Nicholas Cardenas,sony.sth8@gmail.com,21308,8193,41.5026,2.0875
4,380,Samantha Short,sony.shrestha@estudiantat.upc.edu,21308,8193,41.5026,2.0875


In [5]:
df_supermarkets = pd.merge(df_supermarket_products, df_supermarket_location, left_on='store_id', right_on='id', how='left')
df_supermarkets.head()

,store_id,store_name,product_id,product_name,product_price,manufacture_date,expiry_date,quantity,id,commercial_name,county_code,full_address,UTMx,UTMy,latitude,longitude
0,cens-0820210007-3653,ESCLAT-BON PREU,105.1.1,Amul Kool (Kesar),23,01 Mar 2024,28 Aug 2024,21,cens-0820210007-3653,ESCLAT-BON PREU,41.0,"C AMADEU VIVES cant Industria, 08470 Sant Celoni",457873.4,4615087.79,41.686399,2.493811
1,cens-0820210007-3653,ESCLAT-BON PREU,46.1.9,PARLE Krack Jack Crackers Sweet & Salty (Mitha...,4,01 Feb 2024,27 Sep 2024,54,cens-0820210007-3653,ESCLAT-BON PREU,41.0,"C AMADEU VIVES cant Industria, 08470 Sant Celoni",457873.4,4615087.79,41.686399,2.493811
2,cens-0820210007-3653,ESCLAT-BON PREU,33.1.10,RUNutz Almonds (Badam) (1 kg),"1,025",01 Feb 2024,29 Jul 2024,7,cens-0820210007-3653,ESCLAT-BON PREU,41.0,"C AMADEU VIVES cant Industria, 08470 Sant Celoni",457873.4,4615087.79,41.686399,2.493811
3,cens-0820210007-3653,ESCLAT-BON PREU,28.1.17,"Utkal's Kasper HMT Rice (Full Grain, Steam) (...",826,01 Feb 2024,27 Oct 2024,68,cens-0820210007-3653,ESCLAT-BON PREU,41.0,"C AMADEU VIVES cant Industria, 08470 Sant Celoni",457873.4,4615087.79,41.686399,2.493811
4,cens-0820210007-3653,ESCLAT-BON PREU,58.1.15,"Nescafe Classic Coffee Powder, 100% Pure Insta...",17,01 Jul 2023,29 Jan 2025,15,cens-0820210007-3653,ESCLAT-BON PREU,41.0,"C AMADEU VIVES cant Industria, 08470 Sant Celoni",457873.4,4615087.79,41.686399,2.493811


In [6]:
# just for the sake of the query - removing the same locations
df_supermarkets = df_supermarkets.drop_duplicates(subset=['latitude', 'longitude'])

In [7]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

In [8]:
customer_index = 0
current_location = {"lat": df_customer.loc[customer_index]['latitude'], "lon": df_customer.loc[customer_index]['longitude']}

In [9]:
# Calculate distances and store in the DataFrame
df_supermarkets['distance_from_customer'] = df_supermarkets.apply(
    lambda row: haversine(current_location["lat"], current_location["lon"], row['latitude'], row['longitude']),
    axis=1
)

In [10]:
# Sort the DataFrame by distance and select the top 5 closest supermarkets
top_5_closest_supermarkets = df_supermarkets.sort_values(by='distance_from_customer').head(5)
top_5_closest_supermarkets

,store_id,store_name,product_id,product_name,product_price,manufacture_date,expiry_date,quantity,id,commercial_name,county_code,full_address,UTMx,UTMy,latitude,longitude,distance_from_customer
11680,cens-0824440003-296,CONDIS,78.1.19,"Kwality Muesli Fruit & Nut, 76% Multigrains, H...",399,19 Dec 2023,13 Sep 2024,40,cens-0824440003-296,CONDIS,11.0,"AV SANTA COLOMA 64, 08690 Santa Coloma de Cerv...",418124.04,4580088.51,41.368092,2.021002,5.575064
11800,cens-0824440003-313,FAIZY SUPER 2010,66.1.16,"B Natural Guava Juice, Goodness of Fiber, Vit ...",81,01 Feb 2024,27 Oct 2024,84,cens-0824440003-313,FAIZY SUPER 2010,11.0,"C PADRÓ 20, 08690 Santa Coloma de Cervelló",417778.22,4580007.95,41.367331,2.016879,5.869046
11600,cens-0824440003-246,CONSUM,80.1.6,Yogabar Breakfast Protein Bar Pouch (50 g),50,03 Feb 2024,03 Jun 2024,27,cens-0824440003-246,CONSUM,11.0,"AV PAISOS CATALANS 8, 08690 Santa Coloma de Ce...",417736.10,4580109.87,41.368245,2.016361,5.943718
8840,cens-0815800000-632,SUPERMERCAT ALI I NADIA,108.1.13,Nutralite Premium Fat Spread Tub|Enriched with...,280,15 Feb 2024,10 Nov 2024,38,cens-0815800000-632,SUPERMERCAT ALI I NADIA,11.0,"C CARME 27, 08754 PAPIOL",417678.34,4587974.84,41.439074,2.014601,11.442922
8800,cens-0815800000-117,SUPERMERCAT AVILES,3.1.5,Tata Sampann Green Moong Dal (Muga Dali) (Whol...,99,01 Feb 2024,25 Jan 2025,41,cens-0815800000-117,SUPERMERCAT AVILES,11.0,"C ANSELM CLAVE 2, 08754 Papiol",417175.77,4587728.91,41.436808,2.008620,11.488250
